In [1]:
# utils_pump.py
# hang
# GPIO引脚、机械抓相关函数
print('导入机械抓控制模块')
import time
import math
import smbus

class PCA9685:

    def __init__(self, address=0x40, debug=False):
        self.bus = smbus.SMBus(1)  # I2C总线1
        self.address = address
        self.debug = debug
        self.write(0x00, 0x00)  # 初始化PCA9685
  
    def write(self, reg, value):
        self.bus.write_byte_data(self.address, reg, value)
    
    def setPWMFreq(self, freq):
        prescaleval = 25000000.0
        prescaleval /= 4096.0
        prescaleval /= float(freq)
        prescaleval -= 1.0
        prescale = math.floor(prescaleval + 0.5)

        oldmode = self.bus.read_byte_data(self.address, 0x00)
        newmode = (oldmode & 0x7F) | 0x10
        self.write(0x00, newmode)
        self.write(0xFE, int(prescale))
        self.write(0x00, oldmode)
        time.sleep(0.005)
        self.write(0x00, oldmode | 0x80)

    def setPWM(self, channel, on, off):
        self.write(0x06 + 4 * channel, on & 0xFF)
        self.write(0x07 + 4 * channel, on >> 8)
        self.write(0x08 + 4 * channel, off & 0xFF)
        self.write(0x09 + 4 * channel, off >> 8)
    
    def setServoPulse(self, channel, pulse):
        pulse = pulse * 4096 / 20000  # 将脉冲转换为12位的值
        self.setPWM(channel, 0, int(pulse))

# 夹具控制相关函数
def clamp_close(pwm):
    '''
    夹具夹住物体
    '''
    print('    夹具夹住物体')
    pwm.setServoPulse(5, 2100)  # 设置PWM为2100，夹住物体
    time.sleep(1)  # 等待动作完成

def clamp_open(pwm):
    '''
    夹具松开物体
    '''
    print('夹具松开物体')
    pwm.setServoPulse(5, 1500)  # 设置PWM为1500，松开物体
    time.sleep(1)  # 等待动作完成

# 机械臂舵机6控制
if __name__ == '__main__':
    pwm = PCA9685(0x40, debug=False)
    pwm.setPWMFreq(50)  # 设置PWM频率为50Hz

    try:
        while True:
            command = input("输入 'c' 夹紧, 'o' 松开, 'q' 退出: ")
            if command == 'c':
                clamp_close(pwm)
            elif command == 'o':
                clamp_open(pwm)
            elif command == 'q':
                print("程序结束")
                break
            else:
                print("无效输入")
    except KeyboardInterrupt:
        print("程序中断，舵机停止")


导入机械抓控制模块


ModuleNotFoundError: No module named 'smbus'